# Observations

### Input file format
- Each file represents one image from OIDv4, the filename is the same as the image name
- One text file can have multiple bounding boxes
- The coordinates are in the format as follows (class name, left upper corner (x), left upper corner (y), right lower corner (x), right lower corner (y))

In [1]:
!cat oid-samples/2558dd5cb4424643.txt

French fries 80.743424 110.21311999999999 952.539136 933.18656


In [7]:
!cat oid-samples/2c48da973b97f97d.txt

French fries 0.003072 94.44096 390.41024 380.403456
French fries 180.13183999999998 254.459136 890.2789119999999 656.116992


### Output file format 
- File is in XML format
- folder is class name
- path is arbituary, but should have the filename
- size has dimensions
- multiple objects can be represented by <object> tags
- object has a different value expectation than above, it represent the edges of the bounding box, rather than coordinates of top left and bottom right corners, it also is an integer rather than floating point

In [8]:
!cat output-format/20190220_160312.xml

<annotation>
	<folder>goat</folder>
	<filename>20190220_160312.jpg</filename>
	<path>C:\Users\romolo.rocchi\Desktop\goat\20190220_160312.jpg</path>
	<source>
		<database>Unknown</database>
	</source>
	<size>
		<width>640</width>
		<height>480</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>goat</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>102</xmin>
			<ymin>186</ymin>
			<xmax>226</xmax>
			<ymax>263</ymax>
		</bndbox>
	</object>
	<object>
		<name>goat</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>436</xmin>
			<ymin>186</ymin>
			<xmax>567</xmax>
			<ymax>299</ymax>
		</bndbox>
	</object>
</annotation>


# Creating the template
Need to have two templates, master and object level (bounding box level)

In [68]:
master_template = """
<annotation>
    <folder>{class_name}</folder>
    <filename>{filename}</filename>
    <path>/path/to/the/file/{filename}</path>
    <source>
        <database>Unknown</database>
    </source>
    <size>
        <width>{image_width}</width>
        <height>{image_height}</height>
        <depth>3</depth>
    </size>
    <segmented>0</segmented>
    { objects }
</annotation>
"""

object_template ="""<object>
        <name>{class_name}</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>{xmin}</xmin>
            <ymin>{ymin}</ymin>
            <xmax>{xmax}</xmax>
            <ymax>{ymax}</ymax>
        </bndbox>
</object>
"""

In [13]:
import os
from PIL import Image

In [27]:
IMG_DIR = './oid-samples'
LABEL_DIR = './oid-samples/Label/'
CLASS = 'French fries'

In [28]:
data = []

for f in os.listdir(IMG_DIR):
    if f.endswith('.jpg'):
        filename_short = f.split('.')[0]
        data.append({'class': CLASS,
                   'image-path': os.path.join(IMG_DIR, f), 
                   'label-path': os.path.join(LABEL_DIR, filename_short + '.txt')})

In [29]:
data

[{'class': 'French fries',
  'image-path': './oid-samples/6d9fbc51a1948a40.jpg',
  'label-path': './oid-samples/Label/6d9fbc51a1948a40.txt'},
 {'class': 'French fries',
  'image-path': './oid-samples/9ae92a954c81c9f6.jpg',
  'label-path': './oid-samples/Label/9ae92a954c81c9f6.txt'},
 {'class': 'French fries',
  'image-path': './oid-samples/75d869be38b6c2e7.jpg',
  'label-path': './oid-samples/Label/75d869be38b6c2e7.txt'},
 {'class': 'French fries',
  'image-path': './oid-samples/9984cd28f61bfa1b.jpg',
  'label-path': './oid-samples/Label/9984cd28f61bfa1b.txt'},
 {'class': 'French fries',
  'image-path': './oid-samples/f39df149a3d682a1.jpg',
  'label-path': './oid-samples/Label/f39df149a3d682a1.txt'},
 {'class': 'French fries',
  'image-path': './oid-samples/93ff8fc106d9e401.jpg',
  'label-path': './oid-samples/Label/93ff8fc106d9e401.txt'},
 {'class': 'French fries',
  'image-path': './oid-samples/f3fe70d0a2f39fb0.jpg',
  'label-path': './oid-samples/Label/f3fe70d0a2f39fb0.txt'},
 {'cla

In [30]:
OUTPUT_FOLDER = './output'

In [64]:
def return_boxes_from_lines(lines):
    boxes = []
    for l in lines:
        box = []
        l = l.replace('\n', '')
        items = l.split()
        for i in items:
            if '.' in i:
                box.append(i)
        boxes.append(box)
    return boxes

In [76]:
master_template = """
<annotation>
    <folder>{class_name}</folder>
    <filename>{filename}</filename>
    <path>/path/to/the/file/{filename}</path>
    <source>
        <database>Unknown</database>
    </source>
    <size>
        <width>{image_width}</width>
        <height>{image_height}</height>
        <depth>3</depth>
    </size>
    <segmented>0</segmented>
    {objects_template}
</annotation>
"""

object_template ="""<object>
        <name>{class_name}</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>{xmin}</xmin>
            <ymin>{ymin}</ymin>
            <xmax>{xmax}</xmax>
            <ymax>{ymax}</ymax>
        </bndbox>
    </object>
"""

### Iterate through item data

In [ ]:
def return_dimensions(image_path):
    
    image = Image.open(image_path)
    height = image.height
    width = image.width
    return height, width

In [75]:
for item in data:
    
    filename = item['image-path'].split('/')[-1]
    filename_short = filename.split('.')[0]    
    
    # Get image dimensions
    def return_dimensions(image_path):
    image = Image.open(item['image-path'])
    height = image.height
    width = image.width
    return height, width
    print('The width and height of {} is {} x {}'.format(item['image-path'], width, height))
    
    # Load Labels
    with open(item['label-path'], 'r') as f:
        lines = f.readlines()
        f.close()
        
    boxes = return_boxes_from_lines(lines)
    
    # Create Objects Template
    objects = ''
    for box in boxes:
        box_template = object_template.format(xmin=int(float(box[0])),
                                             ymin=int(float(box[1])),
                                             xmax=int(float(box[2])),
                                             ymax=int(float(box[3])),
                                             class_name=CLASS)
        objects += box_template
    
    # Creat Master template
    
    main_template = master_template.format(class_name=CLASS,
                                          filename=item['image-path'].split('/')[-1],
                                          image_height=height,
                                          image_width=width,
                                          objects_template=objects)
    
    
    print(main_template)
    # Save to folder
    with open(os.path.join(OUTPUT_FOLDER, filename_short + '.xml'), 'w') as xml_writer:
        xml_writer.write(main_template)
        xml_writer.close()
        
    print('\n')
    

The width and height of ./oid-samples/6d9fbc51a1948a40.jpg is 1024 x 678

<annotation>
    <folder>French fries</folder>
    <filename>6d9fbc51a1948a40.jpg</filename>
    <path>/path/to/the/file/6d9fbc51a1948a40.jpg</path>
    <source>
        <database>Unknown</database>
    </source>
    <size>
        <width>1024</width>
        <height>678</height>
        <depth>3</depth>
    </size>
    <segmented>0</segmented>
    <object>
        <name>French fries</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>0</xmin>
            <ymin>67</ymin>
            <xmax>850</xmax>
            <ymax>535</ymax>
        </bndbox>
</object>

</annotation>



The width and height of ./oid-samples/9ae92a954c81c9f6.jpg is 1024 x 768

<annotation>
    <folder>French fries</folder>
    <filename>9ae92a954c81c9f6.jpg</filename>
    <path>/path/to/the/file/9ae92a954c81c9f6.jpg</path>
    <source>
        <database>Unk